In [1]:
import torch
from torchtext.datasets import IMDB, AG_NEWS, YahooAnswers
from torchtext.vocab import GloVe
from torchtext.data import to_map_style_dataset
from torchtext.data.utils import get_tokenizer
from torch.nn.utils.rnn import pad_sequence
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam
import torch.nn.functional as F
from torch.nn import LSTM, GRU, Linear, Softmax, Conv2d, Dropout
from tqdm import tqdm
!pip install transformers
from transformers import BertTokenizer, BertForSequenceClassification

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
train_set = AG_NEWS(split='train')
test_set = AG_NEWS(split='test')
num_classes = 4

In [4]:
tokenizer = get_tokenizer('basic_english')
embedding = GloVe(name='6B', dim=50)

In [5]:
from torch.utils.data import Dataset

class ClassificationDataset(Dataset):
    def __init__(self, dataset, num_classes, tokenizer):
        self.num_classes = num_classes
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return self.dataset.__len__()

    def __getitem__(self, idx):
        label, text = self.dataset.__getitem__(idx)
        return int(label) - 1, self.tokenizer(text)

In [6]:
def collate_batch(batch):
    label_list, text_list = [], []
    for (_label, _tokens) in batch:
        label_list.append(_label)
        embed = embedding.get_vecs_by_tokens(_tokens)
        text_list.append(embed)
    label_list = torch.tensor(label_list, dtype=torch.int64)
    text_list = pad_sequence(text_list, batch_first=True)
    return label_list.to(device), text_list.to(device)

In [7]:
train_set = to_map_style_dataset(train_set)
test_set = to_map_style_dataset(test_set)

In [8]:
train_set = ClassificationDataset(train_set, num_classes, tokenizer)
test_set = ClassificationDataset(test_set, num_classes, tokenizer)

In [9]:
BATCH_SIZE = 64
SHUFFLE = True
NUM_EPOCHS = 3

In [10]:
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE,
                          collate_fn=collate_batch, shuffle=SHUFFLE)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE,
                         collate_fn=collate_batch, shuffle=SHUFFLE)

In [11]:
def evaluate(model, data_loader, loss=CrossEntropyLoss()):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (labels, text) in enumerate(data_loader):
            predicted_label = model(text)
            loss_ = loss(predicted_label, labels)
            total_acc += (predicted_label.argmax(1) == labels).sum().item()
            total_count += labels.size(0)
    return total_acc / total_count


def train(model, optimizer, train_loader, loss=CrossEntropyLoss(), log_interval=50):
    model.train()
    total_acc, total_count = 0, 0
    pbar = tqdm(total=len(train_loader),
                desc=f'Epoch [{epoch + 1}/{NUM_EPOCHS}]')
    for idx, (labels, text) in enumerate(train_loader):
        output = model(text)
        loss_ = loss(output, labels)
        optimizer.zero_grad()
        loss_.backward()
        optimizer.step()
        total_acc += (output.argmax(1) == labels).sum().item()
        total_count += labels.size(0)
        pbar.update()
        if idx % log_interval == 0 and idx > 0:
            pbar.set_postfix(loss=loss_, accuracy=total_acc / total_count)
            total_acc, total_count = 0, 0

    pbar.close()

In [12]:
class BidirectionalLSTMClassifier(torch.nn.Module):
    def __init__(self, num_classes, hidden_size, num_layers):
        super().__init__()
        self.num_layers = num_layers
        self.LSTM = LSTM(50, hidden_size, num_layers=num_layers,
                         batch_first=True, bidirectional=True)
        self.linear = Linear(2 * hidden_size, num_classes)
        self.softmax = Softmax(dim=1)

    def forward(self, x):
        _, (h_n, _) = self.LSTM(x)
        h_forward = h_n[2 * self.num_layers - 2]
        h_backward = h_n[2 * self.num_layers - 1]
        y = self.linear(torch.cat((h_forward, h_backward), 1))
        return self.softmax(y)

In [13]:
class CNNClassifier(torch.nn.Module):
    def __init__(self, num_classes, in_channels, out_channels, kernel_heights, pad=0, stri=1, embed_dim=50, drop=0.2):
        super().__init__()
        self.conv1 = Conv2d(in_channels, out_channels[0], kernel_size=(kernel_heights[0], embed_dim), stride=stri, padding=pad)
        self.conv2 = Conv2d(in_channels, out_channels[1], kernel_size=(kernel_heights[1], embed_dim), stride=stri, padding=pad)
        self.conv3 = Conv2d(in_channels, out_channels[2], kernel_size=(kernel_heights[2], embed_dim), stride=stri, padding=pad)
        self.drop = Dropout(drop)
        self.fc = Linear(sum(out_channels), num_classes)
        self.soft = Softmax(dim=1)
    
    def _conv_n_maxpool_1d(self, input, conv_layer):

        conved = conv_layer(input) # conved.size() = (batch_size, out_channels[0], dim, 1)
        reld = F.relu(conved.squeeze(3)) # reld.size() = (batch_size, out_channels[0], dim)
        max_out = F.max_pool1d(reld, reld.size()[2]).squeeze(2) # maxpool_out.size() = (batch_size, out_channels[0])

        return max_out

    def forward(self, x):
        # x.size() = (batch_size, num_seq, embed_dim)
        x = x.unsqueeze(1) # x.size() = (batch_size, 1, num_seq, embed_dim)

        out_1 = self._conv_n_maxpool_1d(x, self.conv1)
        out_2 = self._conv_n_maxpool_1d(x, self.conv2)
        out_3 = self._conv_n_maxpool_1d(x, self.conv3)

        cat_out = torch.cat((out_1, out_2, out_3), dim=1)

        drop = self.drop(cat_out)
        fc_out = self.fc(drop)
        out = self.soft(fc_out)

        return out

In [14]:
#model = BidirectionalLSTMClassifier(num_classes, 64, 2).to(device)
model = CNNClassifier(num_classes, in_channels=1, out_channels=(3,5,7), kernel_heights=(2,3,4)).to(device)
optim = Adam(model.parameters())

for epoch in range(3):
    train(model, optim, train_loader)

Epoch [3/3]: 100%|██████████| 1875/1875 [00:30<00:00, 61.91it/s, accuracy=0.873, loss=tensor(0.8291, device='cuda:0', grad_fn=<NllLossBackward0>)]


In [15]:
test_accuracy = evaluate(model, test_loader)
print(f'Test accuracy: {test_accuracy}')

Test accuracy: 0.8871052631578947


In [16]:
!pip install textattack[tensorflow]

In [19]:
import textattack
import torchtext

class CustomPyTorchModelWrapper(textattack.models.wrappers.model_wrapper.ModelWrapper):
    def __init__(self, model, outdim, vocab=torchtext.vocab.GloVe("6B", dim=50), tokenizer=torchtext.data.utils.get_tokenizer("basic_english")):
        self.model = model
        self.tokenizer = tokenizer
        self.outdim = outdim
        self.vocab = vocab
    
    def __call__(self, text_input_list):
        preds = torch.zeros(size=(len(text_input_list),self.outdim))
        for i, review in enumerate(text_input_list):
            tokens = self.tokenizer(review)
            input = self.vocab.get_vecs_by_tokens(tokens)
            with torch.no_grad():
                prediction = self.model(torch.unsqueeze(input,dim=0).to(device))
                preds[i] = prediction

        return preds

In [21]:
model_wrapper = CustomPyTorchModelWrapper(model, outdim=num_classes)

dataset = textattack.datasets.HuggingFaceDataset("ag_news", split="test")
#attack = textattack.attack_recipes.faster_genetic_algorithm_jia_2019.FasterGeneticAlgorithmJia2019.build(model_wrapper)
attack = textattack.attack_recipes.pwws_ren_2019.PWWSRen2019.build(model_wrapper)
attack_args = textattack.AttackArgs(num_examples=100)
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

Using custom data configuration default
Reusing dataset ag_news (/root/.cache/huggingface/datasets/ag_news/default/0.0.0/fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)
textattack: Loading datasets dataset ag_news, split test.
textattack: Unknown if model of class <class '__main__.CNNClassifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.

  0%|          | 0/100 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 




  1%|          | 1/100 [00:00<00:25,  3.92it/s]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [00:00<00:26,  3.73it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Business (100%)]] --> [[World (94%)]]

Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken [[parent]] [[firm]] Federal Mogul.

Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken [[nurture]] [[loyal]] Federal Mogul.





[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   2%|▏         | 2/100 [00:00<00:40,  2.44it/s]
[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   2%|▏         | 2/100 [00:00<00:40,  2.41it/s]

...

--------------------------------------------- Result 98 ---------------------------------------------
[[Business (100%)]] --> [[Sci/tech (67%)]]

Staples Profit Up, to Enter China Market  NEW YORK (Reuters) - Staples Inc. &lt;A HREF="http://www.investor.reuters.com/FullQuote.aspx?ticker=SPLS.O target=/stocks/quickinfo/fullquote"&gt;SPLS.O&lt;/A&gt;, the top U.S.  office [[products]] retailer, on Tuesday reported a 39 percent jump  in quarterly profit, [[raised]] its full-year [[forecast]] and said it  plans to [[enter]] the fast-growing [[Chinese]] [[market]], sending its  shares higher.

Staples Profit Up, to Enter China Market  NEW YORK (Reuters) - Staples Inc. &lt;A HREF="http://www.investor.reuters.com/FullQuote.aspx?ticker=SPLS.O target=/stocks/quickinfo/fullquote"&gt;SPLS.O&lt;/A&gt;, the top U.S.  office [[intersection]] retailer, on Tuesday reported a 39 percent jump  in quarterly profit, [[kindle]] its full-year [[prognosticate]] and said it  plans to [[enrol]] the fast-growin


[Succeeded / Failed / Skipped / Total] 62 / 22 / 15 / 99:  99%|█████████▉| 99/100 [00:52<00:00,  1.89it/s]

--------------------------------------------- Result 99 ---------------------------------------------
[[World (100%)]] --> [[Sci/tech (69%)]]

Delegation [[Is]] [[Delayed]] [[Before]] [[Reaching]] Najaf AGHDAD, Iraq, Aug. 17  A delegation of Iraqis was delayed for security reasons today but [[still]] intended to [[visit]] Najaf to [[try]] to [[convince]] a [[rebellious]] [[Shiite]] [[cleric]] and his militia to [[evacuate]] a shrine in the holy city and end ...

Delegation [[cost]] [[retard]] [[ahead]] [[accomplish]] Najaf AGHDAD, Iraq, Aug. 17  A delegation of Iraqis was delayed for security reasons today but [[soundless]] intended to [[chat]] Najaf to [[endeavour]] to [[convert]] a [[disaffected]] [[Shi'ite]] [[churchman]] and his militia to [[empty]] a shrine in the holy city and end ...





[Succeeded / Failed / Skipped / Total] 62 / 22 / 15 / 99: 100%|██████████| 100/100 [00:53<00:00,  1.88it/s]
[Succeeded / Failed / Skipped / Total] 63 / 22 / 15 / 100: 100%|██████████| 100/100 [00:53<00:00,  1.88it/s]

--------------------------------------------- Result 100 ---------------------------------------------
[[Business (100%)]] --> [[Sci/tech (68%)]]

Consumer Prices [[Down]], Industry Output [[Up]]  WASHINGTON (Reuters) - [[U]].S. consumer prices [[dropped]] in July  for the [[first]] [[time]] in eight months as a sharp run up in energy  costs reversed, the government [[said]] in a report that suggested  a slow rate of interest rate hikes is likely.

Consumer Prices [[kill]], Industry Output [[upwardly]]  WASHINGTON (Reuters) - [[uranium]].S. consumer prices [[overlook]] in July  for the [[kickoff]] [[clock]] in eight months as a sharp run up in energy  costs reversed, the government [[aver]] in a report that suggested  a slow rate of interest rate hikes is likely.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 63     |
| Number of failed attacks:     | 22     |
| Number